In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
df=pd.read_csv('/content/gurgaon_properties_post_feature_selection (1).csv')

In [ ]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0,36,3.0,2.0,2,1,850.0,0.0,0.0,0.0,1,1,0.82
1,0,95,2.0,2.0,2,1,1226.0,1.0,0.0,0.0,1,2,0.95
2,0,103,2.0,2.0,1,1,1000.0,0.0,0.0,0.0,1,0,0.32
3,0,99,3.0,4.0,4,3,1615.0,1.0,0.0,1.0,0,2,1.60
4,0,5,2.0,2.0,1,3,582.0,0.0,1.0,0.0,0,2,0.48


In [ ]:
# prompt: Using dataframe df: write code to make a new column total rooms

# Create a new column 'total_rooms' by summing the number of bedrooms, bathrooms, servant rooms, and store rooms.
df['total_rooms'] = df['bedRoom'] + df['bathroom'] + df['servant room'] + df['store room']
# Display the first few rows of the DataFrame to verify the new column.
df.head()


,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price,total_rooms
0,0,36,3.0,2.0,2,1,850.0,0.0,0.0,0.0,1,1,0.82,5.0
1,0,95,2.0,2.0,2,1,1226.0,1.0,0.0,0.0,1,2,0.95,5.0
2,0,103,2.0,2.0,1,1,1000.0,0.0,0.0,0.0,1,0,0.32,4.0
3,0,99,3.0,4.0,4,3,1615.0,1.0,0.0,1.0,0,2,1.60,8.0
4,0,5,2.0,2.0,1,3,582.0,0.0,1.0,0.0,0,2,0.48,5.0


In [ ]:
X = df.drop(columns=['price'])
y = df['price']
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [ ]:
df.head(1)

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price,total_rooms
0,0,36,3.0,2.0,2,1,850.0,0.0,0.0,0.0,1,1,0.82,5.0


In [ ]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room','total_rooms']),
        ('cat', OrdinalEncoder(), columns_to_encode)
    ],
    remainder='passthrough'
)

In [ ]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [ ]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
scores.mean(),scores.std()

(np.float64(0.7363096633436829), np.float64(0.032380057544299276))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room', 'store room',
                                                   'total_rooms']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', LinearRegression())])

In [ ]:
y_pred = pipeline.predict(X_test)
y_pred = np.expm1(y_pred)
mean_absolute_error(np.expm1(y_test),y_pred)


0.9463822160089357

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.1 MB/s eta 0:00:00


In [ ]:
def scorer(model_name, model):

    output = []

    output.append(model_name)

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

    output.append(scores.mean())

    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

    pipeline.fit(X_train,y_train)

    y_pred = pipeline.predict(X_test)

    y_pred = np.expm1(y_pred)

    output.append(mean_absolute_error(np.expm1(y_test),y_pred))

    return output

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor,
    AdaBoostRegressor, BaggingRegressor, StackingRegressor, HistGradientBoostingRegressor
)
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

model_dict = {
    'linear_reg': LinearRegression(),
    'ridge': Ridge(),
    'lasso': Lasso(),
    'elasticnet': ElasticNet(),
    'huber': HuberRegressor(),
    'svr': SVR(),
    'decision_tree': DecisionTreeRegressor(),
    'random_forest': RandomForestRegressor(),
    'extra_trees': ExtraTreesRegressor(),
    'gradient_boosting': GradientBoostingRegressor(),
    'hist_gradient_boosting': HistGradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'bagging': BaggingRegressor(),
    'mlp': MLPRegressor(),
    'gaussian_process': GaussianProcessRegressor(),
    'pls': PLSRegression(),
    'kernel_ridge': KernelRidge(),
    'xgboost': XGBRegressor(),
    'lightgbm': LGBMRegressor(),
    'catboost': CatBoostRegressor(verbose=0)
}


In [ ]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_huber.py:343: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_huber.py:343: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_huber.py:343: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000654 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 428
[LightGBM] [Info] Number of data points in the train set: 3198, number of used features: 13
[LightGBM] [Info] Start training from score 1.048736
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000320 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 428
[LightGBM] [Info] Number of data points in the train set: 3198, number of used features: 13
[LightGBM] [Info] Start training from score 1.044723


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000693 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 430
[LightGBM] [Info] Number of data points in the train set: 3198, number of used features: 13
[LightGBM] [Info] Start training from score 1.043458
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000314 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 429
[LightGBM] [Info] Number of data points in the train set: 3198, number of used features: 13
[LightGBM] [Info] Start training from score 1.048783


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000457 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 428
[LightGBM] [Info] Number of data points in the train set: 3199, number of used features: 13
[LightGBM] [Info] Start training from score 1.041440
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000306 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 430
[LightGBM] [Info] Number of data points in the train set: 3199, number of used features: 13
[LightGBM] [Info] Start training from score 1.049110


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000490 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 429
[LightGBM] [Info] Number of data points in the train set: 3199, number of used features: 13
[LightGBM] [Info] Start training from score 1.044446
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000332 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 430
[LightGBM] [Info] Number of data points in the train set: 3199, number of used features: 13
[LightGBM] [Info] Start training from score 1.043242


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000282 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 426
[LightGBM] [Info] Number of data points in the train set: 2843, number of used features: 13
[LightGBM] [Info] Start training from score 1.047189


In [ ]:
model_output

[['linear_reg', np.float64(0.7363096633436829), 0.9463822160089357],
 ['ridge', np.float64(0.7363125316709096), 0.9463373573598884],
 ['lasso', np.float64(0.05943378064493573), 1.528905986892753],
 ['elasticnet', np.float64(0.06148788092427056), 1.527226313189933],
 ['huber', np.float64(0.7259107039708697), 1.020324512066735],
 ['svr', np.float64(0.7632195577773225), 0.8517966458316863],
 ['decision_tree', np.float64(0.7817522302241031), 0.724795164866303],
 ['random_forest', np.float64(0.880008210974176), 0.5192939817098413],
 ['extra_trees', np.float64(0.866656470432621), 0.5502902187388794],
 ['gradient_boosting', np.float64(0.8725859677433091), 0.5682127993936584],
 ['hist_gradient_boosting',
  np.float64(0.8919680304832672),
  0.5024883700618796],
 ['adaboost', np.float64(0.76290005323493), 0.8321047166451454],
 ['bagging', np.float64(0.8739315308720063), 0.5245850475789531],
 ['mlp', np.float64(0.8035896429934063), 0.7084827782713988],
 ['gaussian_process', np.float64(-24.2691010

In [ ]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])
model_df.sort_values(['mae'])


,name,r2,mae
19,catboost,0.896338,4.867760e-01
10,hist_gradient_boosting,0.891968,5.024884e-01
18,lightgbm,0.891319,5.070194e-01
17,xgboost,0.890685,5.183408e-01
7,random_forest,0.880008,5.192940e-01
12,bagging,0.873932,5.245850e-01
8,extra_trees,0.866656,5.502902e-01
9,gradient_boosting,0.872586,5.682128e-01
13,mlp,0.803590,7.084828e-01
6,decision_tree,0.781752,7.247952e-01


In [ ]:
df.head(3)

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price,total_rooms
0,0,36,3.0,2.0,2,1,850.0,0.0,0.0,0.0,1,1,0.82,5.0
1,0,95,2.0,2.0,2,1,1226.0,1.0,0.0,0.0,1,2,0.95,5.0
2,0,103,2.0,2.0,1,1,1000.0,0.0,0.0,0.0,1,0,0.32,4.0


##OneHotEncoding

In [ ]:
columns_to_encode = ['property_type']

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

# Define column groups
numeric_features = ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room', 'total_rooms']
ordinal_features = columns_to_encode  # e.g., ['property_type']
onehot_features = ['sector', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category', 'balcony']

# Creating the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat_ord', OrdinalEncoder(), ordinal_features),
        ('cat_ohe', OneHotEncoder(drop='first', sparse_output=False), onehot_features)
    ],
    remainder='passthrough'
)



In [ ]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
scores.mean()

np.float64(0.8558122985956507)

In [ ]:
scores.std()

np.float64(0.015558218710840465)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room', 'store room',
                                                   'total_rooms']),
                                                 ('cat_ord', OrdinalEncoder(),
                                                  ['property_type']),
                                                 ('cat_ohe',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['sector', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category',
                                                   'balcony'])])),
                ('regressor', LinearRegression())])

In [ ]:
y_pred = pipeline.predict(X_test)
y_pred = np.expm1(y_pred)
mean_absolute_error(np.expm1(y_test),y_pred)

0.648387945111214

In [ ]:
def scorer(model_name, model):

    output = []

    output.append(model_name)

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

    output.append(scores.mean())

    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

    pipeline.fit(X_train,y_train)

    y_pred = pipeline.predict(X_test)

    y_pred = np.expm1(y_pred)

    output.append(mean_absolute_error(np.expm1(y_test),y_pred))

    return output




In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor,
    AdaBoostRegressor, BaggingRegressor, StackingRegressor, HistGradientBoostingRegressor
)
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

model_dict = {
    'linear_reg': LinearRegression(),
    'ridge': Ridge(),
    'lasso': Lasso(),
    'elasticnet': ElasticNet(),
    'huber': HuberRegressor(),
    'svr': SVR(),
    'decision_tree': DecisionTreeRegressor(),
    'random_forest': RandomForestRegressor(),
    'extra_trees': ExtraTreesRegressor(),
    'gradient_boosting': GradientBoostingRegressor(),
    'hist_gradient_boosting': HistGradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'bagging': BaggingRegressor(),
    'mlp': MLPRegressor(),
    'gaussian_process': GaussianProcessRegressor(),
    'pls': PLSRegression(),
    'kernel_ridge': KernelRidge(),
    'xgboost': XGBRegressor(),
    'lightgbm': LGBMRegressor(),
    'catboost': CatBoostRegressor(verbose=0)
}

In [ ]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_huber.py:343: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_huber.py:343: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_huber.py:343: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000480 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 443
[LightGBM] [Info] Number of data points in the train set: 3198, number of used features: 75
[LightGBM] [Info] Start training from score 1.048736
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000369 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 449
[LightGBM] [Info] Number of data points in the train set: 3198, number of used features: 78
[LightGBM] [Info] Start training from score 1.044723


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000444 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 452
[LightGBM] [Info] Number of data points in the train set: 3198, number of used features: 79
[LightGBM] [Info] Start training from score 1.043458
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000372 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 450
[LightGBM] [Info] Number of data points in the train set: 3198, number of used features: 78
[LightGBM] [Info] Start training from score 1.048783


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000361 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 445
[LightGBM] [Info] Number of data points in the train set: 3199, number of used features: 76
[LightGBM] [Info] Start training from score 1.041440
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000370 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 450
[LightGBM] [Info] Number of data points in the train set: 3199, number of used features: 78
[LightGBM] [Info] Start training from score 1.049110


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000353 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 447
[LightGBM] [Info] Number of data points in the train set: 3199, number of used features: 77
[LightGBM] [Info] Start training from score 1.044446
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000407 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 446
[LightGBM] [Info] Number of data points in the train set: 3199, number of used features: 76
[LightGBM] [Info] Start training from score 1.043242


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000320 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 436
[LightGBM] [Info] Number of data points in the train set: 2843, number of used features: 72
[LightGBM] [Info] Start training from score 1.047189


In [ ]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])
model_df.sort_values(['mae'])

,name,r2,mae
8,extra_trees,0.887847,0.473674
19,catboost,0.898583,0.486077
17,xgboost,0.889051,0.500567
13,mlp,0.882347,0.519208
7,random_forest,0.874455,0.522761
5,svr,0.883607,0.529006
10,hist_gradient_boosting,0.870676,0.547939
18,lightgbm,0.871884,0.556468
12,bagging,0.865910,0.559017
9,gradient_boosting,0.856486,0.596090


## Target Encoder

In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.0 MB/s eta 0:00:00


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
import category_encoders as ce

# Define column groups
numeric_cols = ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room', 'total_rooms']
ordinal_cols = ['property_type', 'balcony', 'furnishing_type', 'luxury_category', 'floor_category']  # ordinal relationship assumed
onehot_cols = ['agePossession']  # nominal, not ordinal
target_encode_cols = ['sector']  # high cardinality categorical feature

# Build column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('ord', OrdinalEncoder(), ordinal_cols),
        ('ohe', OneHotEncoder(drop='first', sparse_output=False), onehot_cols),
        ('target', ce.TargetEncoder(), target_encode_cols)
    ],
    remainder='passthrough'
)


In [ ]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
scores.mean(),scores.std()

(np.float64(0.8289972166194003), np.float64(0.018959795853286946))

In [ ]:
def scorer(model_name, model):

    output = []

    output.append(model_name)

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

    output.append(scores.mean())

    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

    pipeline.fit(X_train,y_train)

    y_pred = pipeline.predict(X_test)

    y_pred = np.expm1(y_pred)

    output.append(mean_absolute_error(np.expm1(y_test),y_pred))

    return output


In [ ]:

X['sector'] = X['sector'].astype('object')

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor,
    AdaBoostRegressor, BaggingRegressor, StackingRegressor, HistGradientBoostingRegressor
)
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

model_dict = {
    'linear_reg': LinearRegression(),
    'ridge': Ridge(),
    'lasso': Lasso(),
    'elasticnet': ElasticNet(),
    'huber': HuberRegressor(),
    'svr': SVR(),
    'decision_tree': DecisionTreeRegressor(),
    'random_forest': RandomForestRegressor(),
    'extra_trees': ExtraTreesRegressor(),
    'gradient_boosting': GradientBoostingRegressor(),
    'hist_gradient_boosting': HistGradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'bagging': BaggingRegressor(),
    'mlp': MLPRegressor(),
    'gaussian_process': GaussianProcessRegressor(),
    'pls': PLSRegression(),
    'kernel_ridge': KernelRidge(),
    'xgboost': XGBRegressor(),
    'lightgbm': LGBMRegressor(),
    'catboost': CatBoostRegressor(verbose=0)
}


In [ ]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])
model_df.sort_values(['mae'])


In [ ]:
pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.5 MB/s eta 0:00:00


In [ ]:
import optuna
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error
import numpy as np

# Define your input and target variables (ensure these are defined)
# X, y = ...

# Define model_dict with all models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor,
    AdaBoostRegressor, BaggingRegressor, StackingRegressor, HistGradientBoostingRegressor
)
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

model_dict = {
    'linear_reg': LinearRegression(),
    'ridge': Ridge(),
    'lasso': Lasso(),
    'elasticnet': ElasticNet(),
    'huber': HuberRegressor(),
    'svr': SVR(),
    'decision_tree': DecisionTreeRegressor(),
    'random_forest': RandomForestRegressor(),
    'extra_trees': ExtraTreesRegressor(),
    'gradient_boosting': GradientBoostingRegressor(),
    'hist_gradient_boosting': HistGradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'bagging': BaggingRegressor(),
    'mlp': MLPRegressor(),
    'gaussian_process': GaussianProcessRegressor(),
    'pls': PLSRegression(),
    'kernel_ridge': KernelRidge(),
    'xgboost': XGBRegressor(),
    'lightgbm': LGBMRegressor(),
    'catboost': CatBoostRegressor(verbose=0)
}

# Define hyperparameter tuning for selected models
def get_model_with_params(model_name, trial):
    if model_name == 'ridge':
        return Ridge(alpha=trial.suggest_float('alpha', 1e-3, 100.0, log=True))
    elif model_name == 'lasso':
        return Lasso(alpha=trial.suggest_float('alpha', 1e-3, 100.0, log=True))
    elif model_name == 'elasticnet':
        return ElasticNet(alpha=trial.suggest_float('alpha', 1e-3, 10.0), l1_ratio=trial.suggest_float('l1_ratio', 0.1, 1.0))
    elif model_name == 'svr':
        return SVR(C=trial.suggest_float('C', 0.1, 100.0), epsilon=trial.suggest_float('epsilon', 0.01, 1.0))
    elif model_name == 'random_forest':
        return RandomForestRegressor(n_estimators=trial.suggest_int('n_estimators', 100, 1000), max_depth=trial.suggest_int('max_depth', 3, 20))
    elif model_name == 'xgboost':
        return XGBRegressor(n_estimators=trial.suggest_int('n_estimators', 100, 1000), learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3), max_depth=trial.suggest_int('max_depth', 3, 10))
    elif model_name == 'lightgbm':
        return LGBMRegressor(n_estimators=trial.suggest_int('n_estimators', 100, 1000), learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3), num_leaves=trial.suggest_int('num_leaves', 20, 150))
    elif model_name == 'catboost':
        return CatBoostRegressor(iterations=trial.suggest_int('iterations', 100, 1000), learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3), depth=trial.suggest_int('depth', 3, 10), verbose=0)
    return model_dict[model_name]

# Scorer function with CV evaluation
def scorer(model_name, model):
    print(f"Running {model_name}...")
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    r2_scores = cross_val_score(pipeline, X, y, cv=kf, scoring='r2')
    mae_scores = -cross_val_score(pipeline, X, y, cv=kf, scoring='neg_mean_absolute_error')
    return [model_name, np.mean(r2_scores), np.mean(mae_scores)]

# Tuning + scoring for all models
model_output = []

for model_name in model_dict.keys():
    print(f"Tuning {model_name}...")
    def objective(trial):
        model = get_model_with_params(model_name, trial)
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        score = cross_val_score(pipeline, X, y, cv=kf, scoring='r2').mean()
        return score

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=30)

    best_model = get_model_with_params(model_name, study.best_trial)
    model_output.append(scorer(model_name, best_model))

# Create and sort results DataFrame
model_df = pd.DataFrame(model_output, columns=['name', 'r2', 'mae'])
model_df = model_df.sort_values(by='mae')
print(model_df)


[I 2025-05-08 05:34:02,702] A new study created in memory with name: no-name-71e5b776-de23-4e81-af08-0f5ad5dc9c85


Tuning linear_reg...


[I 2025-05-08 05:34:02,980] Trial 0 finished with value: 0.673341281386876 and parameters: {}. Best is trial 0 with value: 0.673341281386876.
[I 2025-05-08 05:34:03,370] Trial 1 finished with value: 0.673341281386876 and parameters: {}. Best is trial 0 with value: 0.673341281386876.
[I 2025-05-08 05:34:03,921] Trial 2 finished with value: 0.673341281386876 and parameters: {}. Best is trial 0 with value: 0.673341281386876.
[I 2025-05-08 05:34:04,202] Trial 3 finished with value: 0.673341281386876 and parameters: {}. Best is trial 0 with value: 0.673341281386876.
[I 2025-05-08 05:34:04,717] Trial 4 finished with value: 0.673341281386876 and parameters: {}. Best is trial 0 with value: 0.673341281386876.
[I 2025-05-08 05:34:05,203] Trial 5 finished with value: 0.673341281386876 and parameters: {}. Best is trial 0 with value: 0.673341281386876.
[I 2025-05-08 05:34:05,827] Trial 6 finished with value: 0.673341281386876 and parameters: {}. Best is trial 0 with value: 0.673341281386876.
[I 202

Running linear_reg...


[I 2025-05-08 05:34:10,337] A new study created in memory with name: no-name-cbe60caa-be39-43da-b532-bc629f36911e
[I 2025-05-08 05:34:10,476] Trial 0 finished with value: 0.6733412918237782 and parameters: {'alpha': 0.0016853302303807343}. Best is trial 0 with value: 0.6733412918237782.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000255 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 436
[LightGBM] [Info] Number of data points in the train set: 2844, number of used features: 13
[LightGBM] [Info] Start training from score 2.413379


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-05-08 06:22:15,114] A new study created in memory with name: no-name-2ffe2942-4bdc-4d43-b8ac-3300872c1ca4


Tuning catboost...


[I 2025-05-08 06:22:31,277] Trial 0 finished with value: 0.8103367637494079 and parameters: {'iterations': 381, 'learning_rate': 0.17229440330722706, 'depth': 10}. Best is trial 0 with value: 0.8103367637494079.
[I 2025-05-08 06:22:32,844] Trial 1 finished with value: 0.8213180611327597 and parameters: {'iterations': 426, 'learning_rate': 0.19024003956065244, 'depth': 3}. Best is trial 1 with value: 0.8213180611327597.
[I 2025-05-08 06:23:14,789] Trial 2 finished with value: 0.8175902824132578 and parameters: {'iterations': 970, 'learning_rate': 0.1440886250116465, 'depth': 10}. Best is trial 1 with value: 0.8213180611327597.
[I 2025-05-08 06:23:31,646] Trial 3 finished with value: 0.8157575505927674 and parameters: {'iterations': 750, 'learning_rate': 0.15170056339983642, 'depth': 9}. Best is trial 1 with value: 0.8213180611327597.
[I 2025-05-08 06:23:37,484] Trial 4 finished with value: 0.8312678492685908 and parameters: {'iterations': 847, 'learning_rate': 0.29229789615951923, 'dept

Running catboost...
                      name         r2       mae
17                 xgboost   0.818052  0.522002
19                catboost   0.834642  0.523785
10  hist_gradient_boosting   0.830905  0.536794
18                lightgbm   0.824729  0.543971
7            random_forest   0.801740  0.552670
8              extra_trees   0.780178  0.590978
12                 bagging   0.763037  0.592581
9        gradient_boosting   0.793116  0.627478
5                      svr   0.733699  0.716369
6            decision_tree   0.587815  0.724001
13                     mlp   0.748926  0.759805
4                    huber   0.643998  0.884606
2                    lasso   0.673722  0.945644
3               elasticnet   0.673606  0.951748
1                    ridge   0.673466  0.952880
0               linear_reg   0.673341  0.953447
15                     pls   0.650686  1.000462
11                adaboost   0.524106  1.499123
16            kernel_ridge  -0.124501  2.530481
14        gaussian_p

In [ ]:
import optuna
from catboost import CatBoostRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessor setup
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room', 'total_rooms']),
    ('ord', OrdinalEncoder(), ['property_type', 'furnishing_type', 'luxury_category', 'floor_category']),
    ('onehot', OneHotEncoder(drop='first', sparse_output=False), ['agePossession', 'balcony'])
], remainder='passthrough')

# Optuna objective function
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.8),
        'depth': trial.suggest_int('depth', 4, 15),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 15),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 2),
        'random_strength': trial.suggest_float('random_strength', 0, 15),
        'verbose': 0,
        'random_state': 42
    }

    model = CatBoostRegressor(**params)

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    return mean_absolute_error(y_test, y_pred)

# Run Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

# Best model training
best_params = study.best_params
best_params['verbose'] = 0
best_params['random_state'] = 42
best_model = CatBoostRegressor(**best_params)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', best_model)
])

pipeline.fit(X_train, y_train)

# Evaluation
y_pred = pipeline.predict(X_test)
print("Best Parameters:", best_params)
print("R² Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))


[I 2025-05-08 06:40:07,826] A new study created in memory with name: no-name-8cb09fd7-077c-4b7a-b643-fa40f251fb31
[I 2025-05-08 06:40:08,904] Trial 0 finished with value: 0.5757760648193423 and parameters: {'iterations': 398, 'learning_rate': 0.48897847490628876, 'depth': 8, 'l2_leaf_reg': 7.764572766349854, 'bagging_temperature': 0.8942442592803701, 'random_strength': 12.449188670193129}. Best is trial 0 with value: 0.5757760648193423.
[I 2025-05-08 06:40:11,309] Trial 1 finished with value: 0.5353272241640271 and parameters: {'iterations': 554, 'learning_rate': 0.2647918296670395, 'depth': 9, 'l2_leaf_reg': 9.564923870692843, 'bagging_temperature': 0.8028694481975147, 'random_strength': 9.455461176487791}. Best is trial 1 with value: 0.5353272241640271.
[I 2025-05-08 06:40:47,602] Trial 2 finished with value: 0.5305813577362634 and parameters: {'iterations': 683, 'learning_rate': 0.17660797808777856, 'depth': 13, 'l2_leaf_reg': 10.060785948363252, 'bagging_temperature': 0.96105902704

Best Parameters: {'iterations': 990, 'learning_rate': 0.10984836725050896, 'depth': 7, 'l2_leaf_reg': 2.2730726546316955, 'bagging_temperature': 0.007234764040106978, 'random_strength': 2.333094251497335, 'verbose': 0, 'random_state': 42}
R² Score: 0.8477639590299443
MAE: 0.5016618955711338


In [ ]:
from catboost import CatBoostRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

# Define preprocessing
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room', 'total_rooms']),
    ('ord', OrdinalEncoder(), ['property_type', 'furnishing_type', 'luxury_category', 'floor_category']),
    ('onehot', OneHotEncoder(drop='first', sparse_output=False), ['agePossession', 'balcony'])
], remainder='passthrough')

# CatBoost model with best parameters (example; tune via Optuna or GridSearch for better results)
model = CatBoostRegressor(
    iterations=1200,
    learning_rate=0.8,
    depth=14,
    l2_leaf_reg=8,
    random_strength=1,
    bagging_temperature=0.5,
    verbose=0,
    random_state=42
)

# Create pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

# Fit model
pipeline.fit(X, y)

# Predict and evaluate
y_pred = pipeline.predict(X)
print("R² Score:", r2_score(y, y_pred))
print("MAE:", mean_absolute_error(y, y_pred))


R² Score: 0.9999283958401497
MAE: 0.003534062777010083


In [ ]:
from catboost import CatBoostRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import KFold, cross_val_predict
import numpy as np

# Define preprocessing
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room', 'total_rooms']),
    ('ord', OrdinalEncoder(), ['property_type', 'furnishing_type', 'luxury_category', 'floor_category']),
    ('onehot', OneHotEncoder(drop='first', sparse_output=False), ['agePossession', 'balcony'])
], remainder='passthrough')

# CatBoost model with tuned hyperparameters
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.8,
    depth=14,
    l2_leaf_reg=8,
    random_strength=1,
    bagging_temperature=0.5,
    verbose=0,
    random_state=42
)

# Create pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

# Cross-validation setup
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

# Cross-validated predictions
y_pred_cv = cross_val_predict(pipeline, X, y, cv=kfold)

# Final evaluation
r2 = r2_score(y, y_pred_cv)
mae = mean_absolute_error(y, y_pred_cv)

print("Cross-validated R² Score:", r2)
print("Cross-validated MAE:", mae)


Cross-validated R² Score: 0.7663305871499952
Cross-validated MAE: 0.6517741570291758
